In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,5946.0,0.521164,-6.503,0.0,0.0
1,5726.0,0.499457,-6.458,0.0,0.0
2,5347.0,0.462062,-6.253,0.0,0.0
3,5249.0,0.452393,-5.908,0.0,0.0
4,5309.0,0.458313,-5.516,0.0,0.0
...,...,...,...,...,...
43819,6748.0,0.600296,-1.386,0.0,0.0
43820,6429.0,0.568821,-1.661,0.0,0.0
43821,6492.0,0.575037,-1.986,0.0,0.0
43822,6206.0,0.546818,-2.184,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[0]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_AT,Temp_AT
0,5946.0,-6.503
1,5726.0,-6.458
2,5347.0,-6.253
3,5249.0,-5.908
4,5309.0,-5.516
...,...,...
43819,6748.0,-1.386
43820,6429.0,-1.661
43821,6492.0,-1.986
43822,6206.0,-2.184


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_AT,Temp_AT,Lag-2,Lag-7
0,6357.0,-4.576,5220.0,5946.0
1,6360.0,-4.559,5015.0,5726.0
2,6417.0,-4.422,4868.0,5347.0
3,6302.0,-4.344,4974.0,5249.0
4,6601.0,-4.187,4926.0,5309.0
...,...,...,...,...
43651,6748.0,-1.386,7357.0,6158.0
43652,6429.0,-1.661,7000.0,5957.0
43653,6492.0,-1.986,6985.0,6075.0
43654,6206.0,-2.184,6602.0,5748.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0158 - mape: 18160.2871 - mae: 0.0993 - val_loss: 0.0087 - val_mape: 11.8057 - val_mae: 0.0762
Epoch 2/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0080 - mape: 18548.8320 - mae: 0.0684 - val_loss: 0.0049 - val_mape: 8.3512 - val_mae: 0.0533
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0057 - mape: 19113.2734 - mae: 0.0550 - val_loss: 0.0131 - val_mape: 13.9899 - val_mae: 0.0954
Epoch 4/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0050 - mape: 19100.7109 - mae: 0.0507 - val_loss: 0.0108 - val_mape: 13.1051 - val_mae: 0.0882
Epoch 5/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0045 - mape: 19040.2383 - mae: 0.0476 - val_loss: 0.0084 - val_mape: 11.9608 - val_mae: 0.0785
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0043 - mape: 18580.6484 - mae: 0.0459 - val_loss: 0

Epoch 48/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 15534.4434 - mae: 0.0301 - val_loss: 0.0111 - val_mape: 13.7282 - val_mae: 0.0881
Epoch 49/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 16821.4941 - mae: 0.0300 - val_loss: 0.0134 - val_mape: 14.5422 - val_mae: 0.0942
Epoch 50/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 15055.7812 - mae: 0.0300 - val_loss: 0.0109 - val_mape: 13.3315 - val_mae: 0.0843
Epoch 51/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 15098.2383 - mae: 0.0301 - val_loss: 0.0100 - val_mape: 12.6386 - val_mae: 0.0803
Epoch 52/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 15134.7119 - mae: 0.0298 - val_loss: 0.0095 - val_mape: 12.5530 - val_mae: 0.0796
Epoch 53/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0021 - mape: 14727.4355 - mae: 0.0302 - val_l

Epoch 95/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0013 - mape: 6442.2373 - mae: 0.0247 - val_loss: 0.0108 - val_mape: 13.1025 - val_mae: 0.0834
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0014 - mape: 8490.9287 - mae: 0.0255 - val_loss: 0.0105 - val_mape: 12.7555 - val_mae: 0.0821
Epoch 97/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0014 - mape: 4819.1729 - mae: 0.0252 - val_loss: 0.0106 - val_mape: 13.0259 - val_mae: 0.0822
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0013 - mape: 9952.6406 - mae: 0.0249 - val_loss: 0.0099 - val_mape: 12.4718 - val_mae: 0.0787
Epoch 99/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0012 - mape: 4289.6108 - mae: 0.0244 - val_loss: 0.0108 - val_mape: 13.0813 - val_mae: 0.0826
Epoch 100/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0014 - mape: 8084.1885 - mae: 0.0253 - val_loss: 

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 867.9719169139862


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.36989126, 0.34595966, 0.37775493, ..., 0.60349154, 0.5886885 ,
       0.5730443 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.4699556 , 0.45663542, 0.46591021, ..., 0.575037  , 0.54681796,
       0.52323631])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,5427.0,4412.848145
1,5292.0,4170.301270
2,5386.0,4492.546387
3,5848.0,4669.992676
4,6826.0,5053.202637
...,...,...
4363,6748.0,7140.683594
4364,6429.0,6938.542480
4365,6492.0,6780.386719
4366,6206.0,6630.357422


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.081912386467337